In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

from sklearn import model_selection
from sklearn.model_selection import train_test_split
import xgboost as xgb

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.model_selection import GridSearchCV

from joblib import dump, load

pd.options.display.width = 1000 #défaut = 50
pd.options.display.max_columns = 50 #défaut = 20
pd.options.display.max_rows = 100 #défaut = 60
pd.options.display.max_colwidth = 100 #défaut = 50 caractères visualisés dans une colonne

In [7]:
# ****** import des données rééquilibrées par méthode SMOTE() mais non normalisées par méthode StandardScaler() *****

# Import des données training accident: data
# X_sm = pd.read_csv('X_sm.zip', nrows=30000)
X_sm = pd.read_csv('X_sm.zip', nrows=100000)
display(X_sm)

# Import des données training accident: target
y_sm = np.array(pd.read_csv('y_sm.zip', nrows=100000)).ravel()
y_sm = np.where(y_sm == 1, 0, y_sm)
y_sm = np.where(y_sm == 2, 1, y_sm)
display(y_sm)

# Import des données test accident: data & target
X_test = pd.read_csv('X_test.zip', nrows=100000)
display(X_test)

y_test = np.array(pd.read_csv('y_test.zip', nrows=100000)).ravel()
y_test = np.where(y_test == 1, 0, y_test)
y_test = np.where(y_test == 2, 1, y_test)
display(y_test)

,place,catu,sexe,trajet,locp,actp,etatp,cat_age,secu0,secu1,secu2,secu3,secu4,secu5,secu6,secu7,secu8,secu9,catv,obs,obsm,choc,manv,an,mois,lum,agg,int,atm,col,weekday,hr,catr,circ,vosp,prof,plan,surf,infra,situ
0,2.0,2,1,0.0,0.0,0,0.0,2,0,1,0,0,0,0,0,0,0,0,14.0,2.0,0.0,8.0,1.0,6,8,3,1,1,1.0,7.0,3,17,3.0,2.0,0.0,1.0,2.0,1.0,0.0,3.0
1,2.0,2,1,0.0,0.0,0,0.0,2,0,1,0,0,0,0,0,0,0,0,7.0,4.0,9.0,3.0,1.0,8,10,3,1,1,1.0,6.0,5,11,3.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0
2,1.0,3,2,2.0,1.0,5,1.0,1,0,0,0,0,0,0,0,0,1,0,7.0,0.0,1.0,8.0,1.0,9,9,2,2,1,1.0,6.0,3,17,3.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
3,1.0,1,2,5.0,0.0,0,0.0,5,0,1,0,0,0,0,0,0,0,0,7.0,0.0,0.0,0.0,0.0,16,9,1,1,1,1.0,4.0,3,11,4.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
4,1.0,1,2,0.0,0.0,0,0.0,4,0,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,1.0,1.0,9,9,5,2,2,1.0,1.0,1,17,3.0,2.0,0.0,1.0,1.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.0,1,2,1.0,0.0,0,0.0,3,0,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,3.0,9.0,9,1,1,2,1,1.0,5.0,6,11,4.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0
99996,1.0,1,1,5.0,0.0,0,0.0,6,0,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,3.0,1.0,6,1,1,1,1,1.0,5.0,4,17,3.0,3.0,0.0,1.0,1.0,1.0,3.0,1.0
99997,1.0,1,1,0.0,0.0,0,0.0,1,0,0,1,0,0,0,0,0,0,0,2.0,0.0,0.0,3.0,0.0,12,10,1,2,1,1.0,6.0,3,17,4.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0
99998,1.0,1,1,5.0,0.0,0,0.0,3,0,0,0,0,0,0,0,0,1,0,7.0,0.0,2.0,1.0,1.0,10,10,2,2,1,1.0,1.0,4,17,4.0,2.0,0.0,2.0,1.0,1.0,0.0,1.0


array([1, 1, 1, ..., 1, 0, 1], dtype=int64)

,place,catu,sexe,trajet,locp,actp,etatp,cat_age,secu0,secu1,secu2,secu3,secu4,secu5,secu6,secu7,secu8,secu9,catv,obs,obsm,choc,manv,an,mois,lum,agg,int,atm,col,weekday,hr,catr,circ,vosp,prof,plan,surf,infra,situ
0,1.0,1,2,3.0,0.0,0,0.0,6,0,1,0,0,0,0,0,0,0,0,7.0,0.0,0.0,4.0,2.0,8,4,1,2,1,3.0,2.0,1,11,2.0,1.0,0.0,1.0,1.0,3.0,0.0,1.0
1,1.0,1,1,0.0,0.0,0,0.0,3,0,1,0,0,0,0,0,0,0,0,7.0,0.0,1.0,1.0,1.0,9,12,5,2,2,1.0,6.0,6,17,4.0,2.0,3.0,1.0,1.0,1.0,0.0,1.0
2,1.0,1,1,0.0,0.0,0,0.0,4,0,1,0,0,0,0,0,0,0,0,7.0,2.0,0.0,3.0,0.0,12,3,5,2,1,1.0,7.0,2,7,3.0,3.0,0.0,2.0,3.0,1.0,0.0,3.0
3,1.0,1,2,0.0,-1.0,0,-1.0,6,0,1,0,0,0,1,0,0,0,0,7.0,0.0,2.0,3.0,2.0,2021,5,2,2,2,1.0,3.0,3,21,4.0,3.0,0.0,1.0,1.0,1.0,5.0,8.0
4,1.0,1,1,5.0,0.0,0,0.0,2,0,0,1,0,0,0,0,0,0,0,30.0,0.0,2.0,3.0,3.0,7,6,1,2,3,1.0,6.0,2,11,4.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2.0,2,2,0.0,0.0,0,0.0,4,0,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,4.0,1.0,7,4,1,2,1,1.0,2.0,2,11,4.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
99996,1.0,1,1,3.0,0.0,0,0.0,3,0,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,6.0,15.0,7,9,1,2,1,1.0,3.0,4,17,3.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0
99997,2.0,2,1,0.0,-1.0,0,-1.0,6,0,1,0,0,0,0,0,0,0,0,7.0,0.0,2.0,4.0,2.0,18,7,1,1,1,1.0,4.0,5,11,1.0,3.0,0.0,1.0,1.0,1.0,0.0,3.0
99998,1.0,1,1,9.0,0.0,0,-1.0,3,1,1,0,0,0,0,0,0,0,0,7.0,4.0,0.0,1.0,13.0,2021,8,1,2,2,1.0,6.0,5,11,4.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0


array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [15]:
import sys
print(sys.executable)

types_de_donnees = X_test.dtypes
print(types_de_donnees)

C:\ProgramData\anaconda3\python.exe
place      float64
catu         int64
sexe         int64
trajet     float64
locp       float64
actp         int64
etatp      float64
cat_age      int64
secu0        int64
secu1        int64
secu2        int64
secu3        int64
secu4        int64
secu5        int64
secu6        int64
secu7        int64
secu8        int64
secu9        int64
catv       float64
obs        float64
obsm       float64
choc       float64
manv       float64
an           int64
mois         int64
lum          int64
agg          int64
int          int64
atm        float64
col        float64
weekday      int64
hr           int64
catr       float64
circ       float64
vosp       float64
prof       float64
plan       float64
surf       float64
infra      float64
situ       float64
dtype: object


# Modèle XGBClassifier avec optimisation RandomizedSearchCV()

In [8]:
import time
from joblib import dump, load
# Début du chronomètre
start_time = time.time()


#Création du classificateur et entraînement du modèle sur les données de training
# params = {'booster': 'gbtree', # Le type de booster utilisé (par défaut gbtree)
#           'learning_rate': 0.01, # Contrôle le 'taux d’apprentissage' entre 0 et 1
#           'num_class' : 2, # Nb de classes dans le y du dataset
#           'objective': 'multi:softmax', # Pour la classification multiple en utilisant la fonction softmax. Retourne les labels prédits. 
#           #'tree_method' : 'gpu_hist', # Fonctionnement sur GPU vs CPU (non activé)
#           'seed':42}
xgbcl1 = xgb.XGBClassifier(objective="binary:logistic")

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.01, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgbcl1, param_distributions=params, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X_sm, y_sm)

# Enregistrement du modèle (à faire après l'entraînement)
dump(search, 'XGBClassifier_BestParams.joblib')


# Afficher les meilleurs paramètres et le meilleur score
print("Meilleurs paramètres:", search.best_params_)
print("Meilleur score:", search.best_score_)
print("\n The best estimator across ALL searched params:\n", search.best_estimator_)

# Fin du chronomètre
end_time = time.time()

# Calcul de la durée d'exécution
execution_time = end_time - start_time

# Affichage du temps d'exécution
print("\nTemps d'exécution :", execution_time/60, "minutes")

# Maintenant, on applique le modèle entraîné et optimisé en termes d'hyper paramètre à l'ensemble de test
import time
# Début du chronomètre
start_time = time.time()


# Chargement du modèle entrainé optimisé par GridSearch() (à faire sur l'app Streamlit)
search = load('XGBClassifier_BestParams.joblib') 

# Application du modèle aux données de tests
y_pred = search.predict(X_test)

print('Score de bonne prédiction:', search.score(X_test, y_test))
print('\nCrosstab:\n', pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))

from imblearn.metrics import classification_report_imbalanced, geometric_mean_score

print('\nclassification_report_imbalanced:\n', classification_report_imbalanced(y_test, y_pred))

# Fin du chronomètre
end_time = time.time()

# Calcul de la durée d'exécution
execution_time = end_time - start_time

# Affichage du temps d'exécution
print("Temps d'exécution :", execution_time/60, "minutes")

Fitting 3 folds for each of 200 candidates, totalling 600 fits
Meilleurs paramètres: {'colsample_bytree': 0.7550213529560301, 'gamma': 0.15212112147976886, 'learning_rate': 0.16742692948967136, 'max_depth': 5, 'n_estimators': 124, 'subsample': 0.7164916560792167}
Meilleur score: 0.8096199774621494

 The best estimator across ALL searched params:
 XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7550213529560301, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.15212112147976886, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.16742692948967136, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing

In [9]:
# Affiche des 5 variables les plus importantes

feats = {}
for feature, importance in zip(X_test.columns, search.best_estimator_.feature_importances_):
    feats[feature] = importance 
    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Importance'})
importances.sort_values(by='Importance', ascending=False).head(5)

,Importance
secu1,0.312139
secu2,0.092151
catu,0.062935
secu8,0.061800
agg,0.050905


# Modèle XGBClassifier sans optimisation RandomizedSearchCV()

In [6]:
import time
from joblib import dump, load
# Début du chronomètre
start_time = time.time()


#Création du classificateur et entraînement du modèle sur les données de training
xgbcl1 = xgb.XGBClassifier(   base_score = None, booster=None, callbacks=None,
                              colsample_bylevel = None, colsample_bynode=None,
                              colsample_bytree=0.8045997961875188, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None, feature_types=None,
                              gamma=0.04808827554571038, gpu_id=None, grow_policy=None,
                              importance_type=None, interaction_constraints=None,
                              learning_rate=0.2921569793468812, max_bin=None,
                              max_cat_threshold=None, max_cat_to_onehot=None,
                              max_delta_step=None, max_depth=5, max_leaves=None,
                              min_child_weight=None, monotone_constraints=None,
                              n_estimators=138, n_jobs=None, num_parallel_tree=None,
                              predictor=None,
                              subsample = 0.9746919954946938,
                              random_state=42)

xgbcl1.fit(X_sm, y_sm)

# Enregistrement du modèle (à faire après l'entraînement)
dump(xgbcl1, 'XGBClassifier_Params.joblib')

print('Score de bonne prédiction sur dataset de training:', xgbcl1.score(X_sm, y_sm))


# Fin du chronomètre
end_time = time.time()

# Calcul de la durée d'exécution
execution_time = end_time - start_time

# Affichage du temps d'exécution
print("Temps d'exécution :", execution_time/60, "minutes")

Score de bonne prédiction sur dataset de training: 0.815679
Temps d'exécution : 0.828882118066152 minutes


In [7]:
# Obtenir les hyperparamètres du modèle
hyperparameters = xgbcl1.get_params()

# Afficher les hyperparamètres
for param, value in hyperparameters.items():
    print(param, ":", value)

objective : binary:logistic
use_label_encoder : None
base_score : None
booster : None
callbacks : None
colsample_bylevel : None
colsample_bynode : None
colsample_bytree : 0.8045997961875188
early_stopping_rounds : None
enable_categorical : False
eval_metric : None
feature_types : None
gamma : 0.04808827554571038
gpu_id : None
grow_policy : None
importance_type : None
interaction_constraints : None
learning_rate : 0.2921569793468812
max_bin : None
max_cat_threshold : None
max_cat_to_onehot : None
max_delta_step : None
max_depth : 5
max_leaves : None
min_child_weight : None
missing : nan
monotone_constraints : None
n_estimators : 138
n_jobs : None
num_parallel_tree : None
predictor : None
random_state : 42
reg_alpha : None
reg_lambda : None
sampling_method : None
scale_pos_weight : None
subsample : 0.9746919954946938
tree_method : None
validate_parameters : None
verbosity : None


In [9]:
import time
from joblib import dump, load
# Début du chronomètre
start_time = time.time()


# Chargement du modèle (à faire sur l'app Streamlit)
xgbcl1 = load('XGBClassifier_Params.joblib')

# Application du modèle aux données de tests
y_pred = xgbcl1.predict(X_test)

print('Score de bonne prédiction:', xgbcl1.score(X_test, y_test))
print('\nCrosstab:\n', pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))

from imblearn.metrics import classification_report_imbalanced, geometric_mean_score

print('\nclassification_report_imbalanced:\n', classification_report_imbalanced(y_test, y_pred))

# Fin du chronomètre
end_time = time.time()

# Calcul de la durée d'exécution
execution_time = end_time - start_time

# Affichage du temps d'exécution
print("Temps d'exécution :", execution_time, "secondes")

Score de bonne prédiction: 0.81367668817127

Crosstab:
 Classe prédite       0       1
Classe réelle                 
0               158789   45986
1                47329  248719

classification_report_imbalanced:
                    pre       rec       spe        f1       geo       iba       sup

          0       0.77      0.78      0.84      0.77      0.81      0.65    204775
          1       0.84      0.84      0.78      0.84      0.81      0.66    296048

avg / total       0.81      0.81      0.80      0.81      0.81      0.65    500823

Temps d'exécution : 1.5035369396209717 secondes


In [36]:
# Affiche des 5 variables les plus importantes

feats = {}
for feature, importance in zip(X_test.columns, xgbcl1.feature_importances_):
    feats[feature] = importance 
    
importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Importance'})
importances.sort_values(by='Importance', ascending=False).head(5)


,Importance
secu1,0.344368
secu8,0.072027
obs,0.059891
agg,0.055947
secu2,0.055529
